In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

import statsmodels.api as sm
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict

In [16]:
df = pd.read_csv('fiat-punto2.csv', delimiter=';', encoding='cp1254')
df#csv de ; ile ayrıldığı için delimiter ; kullandım.

,fiyat,model,yil,km,vites,yakit,renk,boyaDegisen
0,525000,Grande 1.4 Fire Active,2007,62000,Otomatik,Benzin,Kırmızı,"0 değişen, 0 boyalı"
1,595000,EVO 1.4 Dynamic,2012,115000,Düz,LPG & Benzin,Beyaz,8 boyalı
2,505000,Grande 1.4 Fire S&S,2011,50000,Düz,Benzin,Beyaz,"0 değişen, 0 boyalı"
3,475000,1.3 Multijet Pop,2013,150000,Düz,Dizel,Beyaz,"0 değişen, 0 boyalı"
4,407500,EVO 1.3 Multijet Dynamic,2011,275000,Düz,Dizel,Mavi,"2 değişen, 4 boyalı"
...,...,...,...,...,...,...,...,...
1330,554000,1.4 Popstar S&S,2017,88819,Düz,LPG & Benzin,Beyaz,"0 değişen, 0 boyalı"
1331,580000,1.4 Popstar S&S,2017,44000,Düz,Benzin,Beyaz,"0 değişen, 0 boyalı"
1332,560000,1.4 Popstar S&S,2017,47815,Düz,Benzin,Kırmızı,2 boyalı
1333,585000,1.4 Popstar S&S,2017,53900,Düz,Benzin,Kırmızı,"0 değişen, 0 boyalı"


In [17]:
unwanted_models = ['1.7 D', '1.3 JTD']
df = df[~df['model'].isin(unwanted_models)]
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x) ##verilerin başındaki ve sonundaki boşlukları kaldırdım.

In [18]:
#df['fiyat'] = df['fiyat'].apply(lambda x: x/800000 )
#df['km'] = df['km'].apply(lambda x: x/400000 )

df["boya"]= df["boyaDegisen"]
df['boya'] = df['boya'].apply(lambda x: int(x.split(",")[1].split(" ")[1]) if  x!=None and type(x)!=int  and len(x)>15 else x)
df['boya'] = df['boya'].apply(lambda x: int(x.split(" ")[0]) if x!=None and type(x)!=int  and len(x)<15 and "boyalı" in x else x)

df["Degisen"]= df["boyaDegisen"]
df['Degisen'] = df['Degisen'].apply(lambda x: int(x.split(",")[0].split(" ")[0]) if type(x)!=int and x!=None and len(x)>15 else x)
df['Degisen'] = df['Degisen'].apply(lambda x: int(x.split(" ")[0]) if type(x)!=int and x!=None and len(x)<15 and "değişen" in x else x)
df['Degisen'] = df['Degisen'].apply(lambda x: 0 if type(x)!=int and x!=None and len(x)<15 and "boyalı" in x else x)


df['Degisen'] = df['Degisen'].apply(lambda x: int(x.split(",")[0].split(" ")[0]) if type(x)!=int and x!=None and len(x)>15 else x)

# Değişen Boya bilgisi bulunmayan yani girilmemiş araçların olduğu sütunlar boş olarak değerlendirildi ve mean ile dolduruldu
#mean_value=int(df['Degisen'].mean())
#mean_value2=int(df['boya'].mean())


#df["MotorHacmi"]=df["model"].apply(lambda x:x.split(" ")[0])
#df["MotorHacmi"] = df["model"].apply(lambda x: '.'.join(filter(str.isdigit, x.split(".")[0] + "." + x.split(".")[1]))[:3] if len(x.split(".")) > 1 else None)
#df["model"]=df["model"].apply(lambda x:x.split(" ")[-1])

#df['boya'].fillna(value=int(mean_value2), inplace=True)
#df['Degisen'].fillna(value=int(mean_value), inplace=True)

df=df.drop(["boyaDegisen"],axis=1)
df.head()

,fiyat,model,yil,km,vites,yakit,renk,boya,Degisen
0,525000,Grande 1.4 Fire Active,2007,62000,Otomatik,Benzin,Kırmızı,0,0
1,595000,EVO 1.4 Dynamic,2012,115000,Düz,LPG & Benzin,Beyaz,8,0
2,505000,Grande 1.4 Fire S&S,2011,50000,Düz,Benzin,Beyaz,0,0
3,475000,1.3 Multijet Pop,2013,150000,Düz,Dizel,Beyaz,0,0
4,407500,EVO 1.3 Multijet Dynamic,2011,275000,Düz,Dizel,Mavi,4,2


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1332 entries, 0 to 1334
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   fiyat    1332 non-null   int64 
 1   model    1332 non-null   object
 2   yil      1332 non-null   int64 
 3   km       1332 non-null   int64 
 4   vites    1332 non-null   object
 5   yakit    1332 non-null   object
 6   renk     1332 non-null   object
 7   boya     1332 non-null   int64 
 8   Degisen  1332 non-null   int64 
dtypes: int64(5), object(4)
memory usage: 104.1+ KB


In [20]:
selectedFeatures = ["model", "yil", "km", "vites", "yakit", "renk", "boya", "Degisen"]

# Hedef değişken ve özellikler
y = df["fiyat"]
X = df[selectedFeatures]

# Sayısal ve kategorik özellikleri seçme
numeric_features = ['km','boya','Degisen']
categorical_features = ["yil", "yakit", "vites", "model","renk"]

# Sayısal özellikler için işlem
numeric_transformer = ColumnTransformer(
    transformers=[
        ('scaler', MinMaxScaler(), numeric_features)
    ])

# Kategorik özellikler için işlem
categorical_transformer = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Sayısal ve kategorik özellikleri birleştirme
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Özellikleri işleme
X_preprocessed = preprocessor.fit_transform(X)

# Eğitim ve test veri setlerine ayırma
X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, test_size=0.2, random_state=42)

In [21]:
# Model oluşturma ve eğitme
model = LinearRegression()
model.fit(X_train, y_train)

# Tahmin yapma
y_pred = model.predict(X_test)

# Model performansını değerlendirme
r2_lr_train = model.score(X_train, y_train)
r2_lr_test = model.score(X_test, y_test)
mse_lr = mean_squared_error(y_test, y_pred)
mae_lr = mean_absolute_error(y_test, y_pred)

print("Linear Reg R2 (Eğitim):", r2_lr_train)
print("Linear Reg R2 (Test):", r2_lr_test)
print("Linear Reg MSE:", mse_lr)
print("Linear Ref MAE:", mae_lr)

# Modeli kaydetme
from joblib import dump
dump(model, 'linreg.joblib')

Linear Reg R2 (Eğitim): 0.7415665821954672
Linear Reg R2 (Test): 0.7014542914213657
Linear Reg MSE: 1993123343.8781075
Linear Ref MAE: 32977.38814384939


['linreg.joblib']

In [9]:
from sklearn.ensemble import RandomForestRegressor

# Model oluşturma ve eğitme
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)

# Tahmin yapma
rf_y_pred = rf_model.predict(X_test)

# Model performansını değerlendirme
rf_mae = mean_absolute_error(y_test, rf_y_pred)
print("Random Forest Mean Absolute Error:", rf_mae)

# R2 ve MSE değerleri
r2_rf_train = rf_model.score(X_train, y_train)
r2_rf_test = rf_model.score(X_test, y_test)
mse_rf = mean_squared_error(y_test, rf_y_pred)
mae_rf = mean_absolute_error(y_test, rf_y_pred)
print("Random Forest R2 (Eğitim):", r2_rf_train)
print("Random Forest R2 (Test):", r2_rf_test)
print("Random Forest MSE:", mse_rf)
print("Random Forest MAE:", mae_rf)

# Modeli kaydetme
from joblib import dump
dump(rf_model, 'ranforeg.joblib')

Random Forest Mean Absolute Error: 33161.4862243624
Random Forest R2 (Eğitim): 0.9513826273994155
Random Forest R2 (Test): 0.7168076930162958
Random Forest MSE: 1890622379.2101448
Random Forest MAE: 33161.4862243624


['ranforeg.joblib']

In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error
from joblib import dump

# SVM için parametre ayarı yapma
param_grid = {'C': [0.1, 1, 10, 100], 'gamma': [0.01, 0.1, 1, 10]}
grid_search = GridSearchCV(SVR(kernel='rbf'), param_grid, cv=3)
grid_search.fit(X_train, y_train)

# En iyi parametreleri ve modeli alıyoruz
best_params = grid_search.best_params_
best_svm_model = grid_search.best_estimator_

# En iyi modeli orijinal veri ile tekrar eğitme
best_svm_model.fit(X_train, y_train)

# Tahmin yapma
y_pred_svm_best = best_svm_model.predict(X_test)

# Model performansını değerlendirme
r2_svm_train_best = best_svm_model.score(X_train, y_train)
r2_svm_test_best = best_svm_model.score(X_test, y_test)
mse_svm_best = mean_squared_error(y_test, y_pred_svm_best)
mae_svm_best = mean_absolute_error(y_test, y_pred_svm_best)

print("Best SVM R2 (Eğitim):", r2_svm_train_best)
print("Best SVM R2 (Test):", r2_svm_test_best)
print("Best SVM MSE:", mse_svm_best)
print("Best SVM MAE:", mae_svm_best)

# Modeli kaydetme
dump(best_svm_model, 'svmreg_best.joblib')



Best SVM R2 (Eğitim): 0.030167556213941626
Best SVM R2 (Test): 0.007116722051882518
Best SVM MSE: 6628595830.254158
Best SVM MAE: 65667.88563618106


['svmreg_best.joblib']

In [ ]:
from joblib import load
import pandas as pd

# Modeli yükle
modelName = 'linreg.joblib'
#modelName = 'ranforeg.joblib'
#modelName = 'svmreg.joblib'
clf = load(modelName)

def predict():
    # Kullanıcı girdilerini al
    year_value = int(input("Yıl: "))
    km_value = int(input("Kilometre: "))
    body_value = input("Boya Durumu: ")
    changed_value = input("Değişen Parça Durumu: ")
    gas_value = input("Yakıt Tipi: ")
    gear_value = input("Vites Tipi: ")
    model_value = input("Araç Modeli: ")
    color_value = input("Renk: ")

    # Kullanıcı girdilerinden bir DataFrame oluştur
    myCar = pd.DataFrame({
        'yil': [year_value],
        'km': [km_value],
        'boya': [body_value],
        'Degisen': [changed_value],
        'yakit': [gas_value],
        'vites': [gear_value],
        'model': [model_value],
        'renk': [color_value]
    })

    # Özellikleri işleme
    X_pred_processed = preprocessor.transform(myCar)

    # Modeli kullanarak tahmin yap
    preds = clf.predict(X_pred_processed)

    # Tahmin sonucunu yazdır
    print("Tahmini Fiyat: ", int(preds[0]))

# Tahmin fonksiyonunu çağırın
predict()


In [14]:
y_test_pred_lr = model.predict(X_test)
y_test_pred_rf = rf_model.predict(X_test)

# Tahmin edilen ve gerçek değerleri birleştirin
result_df = pd.DataFrame({'Gerçek Değerler': y_test, 'Linear Regresyon Tahminleri': y_test_pred_lr, 'RFR Tahminleri': y_test_pred_rf})

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# DataFrame'i yazdırın
print(result_df)

# Ayarları geri alın
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')

      Gerçek Değerler  Linear Regresyon Tahminleri  RFR Tahminleri
1283           383000                425667.854129   421349.000000
1064           530000                560632.888039   563897.490000
1253           438000                448620.367064   447770.000000
299            485000                459825.790020   490817.000000
238            465000                436256.245887   476647.500000
590            374900                418699.889269   378027.500000
241            490000                475287.763684   471019.166667
278            445000                410859.098714   435029.000000
890            395000                466569.401895   472475.666667
707            495000                565898.585193   585739.000000
371            505000                523130.231195   524735.000000
332            482500                476519.946434   467603.000000
1300           545000                539489.084162   502827.500000
907            420000                449389.127799   442531.00